## Punto 2

In [35]:
x = 10
from pyomo.environ import *
model = ConcreteModel()

In [36]:
t, w = 3, 5

model.T = RangeSet(t) #Trabajadores
model.W = RangeSet(w) #Trabajos
model.x = Var(model.T, model.W, domain=Binary) #La variable es binaria

Parámetros

In [37]:
# Horas que puede trabajar cada trabajador
h_data = {1: 8, 2: 10, 3: 6}

model.h_disp = Param(model.T, initialize=h_data)

In [38]:
# Ganancia por trabajo
g_data = {1: 50, 2: 60, 3: 40, 4: 70, 5: 30}

model.ganancias = Param(model.W, initialize=g_data)

In [39]:
# Horas para completar un trabajo

hr_data = {1: 4, 2: 5, 3: 3, 4: 6, 5: 2}

model.h_reque = Param(model.W, initialize=hr_data)

In [40]:
for w in model.W:
    print(f"Trabajo {w} tiene {model.h_reque[w]} horas requeridas")

Trabajo 1 tiene 4 horas requeridas
Trabajo 2 tiene 5 horas requeridas
Trabajo 3 tiene 3 horas requeridas
Trabajo 4 tiene 6 horas requeridas
Trabajo 5 tiene 2 horas requeridas


Funcion Objetivo

In [41]:
model.obj = Objective(expr = sum(model.ganancias[t] * model.x[t, w] for t in model.T for w in model.W),
                        sense = maximize)

Restricciones

In [42]:
def restriccion_trabajo(model, w):
    return sum(model.x[t, w] for t in model.T) <= 1

model.restriccion_un_trabajador = Constraint(model.W, rule=restriccion_trabajo)

In [43]:
model.restriccion_un_trabajador.pprint()

restriccion_un_trabajador : Size=5, Index=W, Active=True
    Key : Lower : Body                     : Upper : Active
      1 :  -Inf : x[1,1] + x[2,1] + x[3,1] :   1.0 :   True
      2 :  -Inf : x[1,2] + x[2,2] + x[3,2] :   1.0 :   True
      3 :  -Inf : x[1,3] + x[2,3] + x[3,3] :   1.0 :   True
      4 :  -Inf : x[1,4] + x[2,4] + x[3,4] :   1.0 :   True
      5 :  -Inf : x[1,5] + x[2,5] + x[3,5] :   1.0 :   True


In [44]:
def restriccion_horario(model, t):
    return sum(model.x[t, w] * model.h_reque[w] for w in model.W) <= model.h_disp[t]

model.restriccion_horario = Constraint(model.T, rule=restriccion_horario)


In [45]:
model.restriccion_horario.pprint()

restriccion_horario : Size=3, Index=T, Active=True
    Key : Lower : Body                                                 : Upper : Active
      1 :  -Inf : 4*x[1,1] + 5*x[1,2] + 3*x[1,3] + 6*x[1,4] + 2*x[1,5] :   8.0 :   True
      2 :  -Inf : 4*x[2,1] + 5*x[2,2] + 3*x[2,3] + 6*x[2,4] + 2*x[2,5] :  10.0 :   True
      3 :  -Inf : 4*x[3,1] + 5*x[3,2] + 3*x[3,3] + 6*x[3,4] + 2*x[3,5] :   6.0 :   True


Resolver el modelo

In [46]:
from pyomo.opt import SolverFactory
import os, pathlib

exe = pathlib.Path(os.environ["CONDA_PREFIX"]) / "Library" / "bin" / "glpsol.exe"
solver = SolverFactory("glpk", executable=str(exe))
results = solver.solve(model)


In [47]:
for t in model.T :
    for w in model.W:
        print (f" x [{ t } ,{ w }] = { model.x [t, w].value} ")

 x [1 ,1] = 0.0 
 x [1 ,2] = 0.0 
 x [1 ,3] = 0.0 
 x [1 ,4] = 1.0 
 x [1 ,5] = 0.0 
 x [2 ,1] = 0.0 
 x [2 ,2] = 1.0 
 x [2 ,3] = 1.0 
 x [2 ,4] = 0.0 
 x [2 ,5] = 1.0 
 x [3 ,1] = 1.0 
 x [3 ,2] = 0.0 
 x [3 ,3] = 0.0 
 x [3 ,4] = 0.0 
 x [3 ,5] = 0.0 


## Parte B

In [48]:
# Solo el trabajador 1 puede realizar el trabajo 1

def restriccion_trabajador1(model, t):
    if t != 1:
        return model.x[t, 1] == 0
    return Constraint.Skip

model.restriccion_trabajador1 = Constraint(model.T, rule=restriccion_trabajador1)


In [50]:
# El trabajador 2 no puede realizar el trabajo 3

model.restriccion_trabajador2 = Constraint(expr=model.x[2, 3] == 0)

In [52]:
results = solver.solve(model)
for t in model.T :
    for w in model.W:
        print (f" x [{ t } ,{ w }] = { model.x [t, w].value} ")

 x [1 ,1] = 1.0 
 x [1 ,2] = 0.0 
 x [1 ,3] = 1.0 
 x [1 ,4] = 0.0 
 x [1 ,5] = 0.0 
 x [2 ,1] = 0.0 
 x [2 ,2] = 1.0 
 x [2 ,3] = 0.0 
 x [2 ,4] = 0.0 
 x [2 ,5] = 1.0 
 x [3 ,1] = 0.0 
 x [3 ,2] = 0.0 
 x [3 ,3] = 0.0 
 x [3 ,4] = 1.0 
 x [3 ,5] = 0.0 
